In [35]:
!git clone https://github.com/dikey4405/23520728_Lab4_DeepLearning.git

Cloning into '23520728_Lab4_DeepLearning'...
remote: Enumerating objects: 29, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 29 (delta 6), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (29/29), 1.91 MiB | 4.82 MiB/s, done.
Resolving deltas: 100% (6/6), done.


In [36]:
import sys
sys.path.append('/kaggle/working/23520728_Lab4_DeepLearning') 

# Check thư mục
import os
print(os.listdir('/kaggle/working/23520728_Lab4_DeepLearning'))

['.git', 'dataset.py', 'train.py', 'test.py', 'assignments.ipynb', '__pycache__', 'LSTM.py', 'vocab.py', 'README.md', '23520728_Lab4_DeepLearning', 'LSTM_Bai2.py', 'data', 'config', 'LSTM_Bai3.py']


In [37]:
import torch
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [38]:
!pip install evaluate pyyaml tqdm
!pip install protobuf==3.20.3
!pip install rouge_score

In [39]:
%cd 23520728_Lab4_DeepLearning
base_path = os.getcwd()
base_path

/kaggle/working/23520728_Lab4_DeepLearning/23520728_Lab4_DeepLearning/23520728_Lab4_DeepLearning/23520728_Lab4_DeepLearning/23520728_Lab4_DeepLearning


'/kaggle/working/23520728_Lab4_DeepLearning/23520728_Lab4_DeepLearning/23520728_Lab4_DeepLearning/23520728_Lab4_DeepLearning/23520728_Lab4_DeepLearning'

In [40]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm.auto import tqdm 

import evaluate


SETTINGS = {
    "data_dir": "data",  
    "src_lang": "english",
    "tgt_lang": "vietnamese",
    "batch_size": 32,       
    "lr": 0.0001,
    "epochs": 30,           
    "patience": 3,          
    "hidden_dim": 256,
    "num_layers": 3,
    "dropout": 0.1,
    "weight_decay": 0.00001,
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu")
}

print(f"Running on device: {SETTINGS['device']}")

Running on device: cuda


In [41]:
from vocab import Vocabulary
from dataset import TranslationDataset, translation_collate_fn

from LSTM import Seq2Seq
from LSTM_Bai2 import Seq2SeqBahdanau
from LSTM_Bai3 import Seq2SeqLuong

class DirectTrainer:
    def __init__(self, model_name):
        self.model_name = model_name
        self.device = SETTINGS["device"]
        self.metric = evaluate.load("rouge")
        
        # 1. Prepare Data
        self._prepare_data()
        
        # 2. Build Model
        self._build_model()
        
        # 3. Optimizer & Loss
        self.optimizer = optim.Adam(self.model.parameters(), lr=SETTINGS["lr"])
        self.criterion = nn.CrossEntropyLoss(ignore_index=self.vocab.pad_idx)
        
    def _prepare_data(self):
        print(f"[{self.model_name.upper()}] Loading Data...")
        data_dir = SETTINGS["data_dir"]
        
        # Load Vocab
        self.vocab = Vocabulary(
            f"{data_dir}/small-train.json", 
            SETTINGS["src_lang"], 
            SETTINGS["tgt_lang"]
        )
        
        # Load Datasets
        train_ds = TranslationDataset(f"{data_dir}/small-train.json", self.vocab)
        dev_ds   = TranslationDataset(f"{data_dir}/small-dev.json", self.vocab)
        test_ds  = TranslationDataset(f"{data_dir}/small-test.json", self.vocab)
        
        # Dataloaders
        self.train_loader = DataLoader(train_ds, batch_size=SETTINGS["batch_size"], shuffle=True, collate_fn=translation_collate_fn)
        self.dev_loader   = DataLoader(dev_ds, batch_size=SETTINGS["batch_size"], shuffle=False, collate_fn=translation_collate_fn)
        self.test_loader  = DataLoader(test_ds, batch_size=SETTINGS["batch_size"], shuffle=False, collate_fn=translation_collate_fn)
        
    def _build_model(self):
        print(f"[{self.model_name.upper()}] Building Model...")
        if self.model_name == "bai1":
            self.model = Seq2Seq(self.vocab, SETTINGS["hidden_dim"], SETTINGS["num_layers"], SETTINGS["dropout"])
        elif self.model_name == "bai2":
            self.model = Seq2SeqBahdanau(self.vocab, SETTINGS["hidden_dim"], SETTINGS["num_layers"], SETTINGS["dropout"])
        elif self.model_name == "bai3":
            self.model = Seq2SeqLuong(self.vocab, SETTINGS["hidden_dim"], SETTINGS["num_layers"], SETTINGS["dropout"])
        else:
            raise ValueError("Model name must be 'bai1', 'bai2', or 'bai3'")
            
        self.model.to(self.device)

    def train_one_epoch(self):
        self.model.train()
        total_loss = 0
        
        for batch in tqdm(self.train_loader, desc="Training", leave=False):
            src = batch["src"].to(self.device)
            tgt = batch["tgt"].to(self.device)
            src_len = batch["src_len"].to(self.device)
            
            self.optimizer.zero_grad()
            output = self.model(src, src_len, tgt, teacher_forcing=True)
            
            # Flatten
            output = output[:, 1:].reshape(-1, output.shape[-1])
            tgt = tgt[:, 1:].reshape(-1)
            
            loss = self.criterion(output, tgt)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)
            self.optimizer.step()
            
            total_loss += loss.item()
            
        return total_loss / len(self.train_loader)

    @torch.no_grad()
    def evaluate(self, loader):
        self.model.eval()
        total_loss = 0
        preds, refs = [], []
        
        for batch in tqdm(loader, desc="Evaluating", leave=False):
            src = batch["src"].to(self.device)
            tgt = batch["tgt"].to(self.device)
            src_len = batch["src_len"].to(self.device)
            
            # Calc Loss
            out = self.model(src, src_len, tgt, teacher_forcing=True)
            loss = self.criterion(out[:, 1:].reshape(-1, out.shape[-1]), tgt[:, 1:].reshape(-1))
            total_loss += loss.item()
            
            # Inference Text
            pred_ids = self.model.inference(src, src_len)
            for i in range(src.size(0)):
                preds.append(self.vocab.indices_to_text(pred_ids[i], self.vocab.tgt_lang))
                refs.append(self.vocab.indices_to_text(tgt[i], self.vocab.tgt_lang))
                
        rouge = self.metric.compute(predictions=preds, references=refs)
        return total_loss / len(loader), rouge["rougeL"], preds, refs

    def run(self):
        best_score = 0 
        no_improve = 0
        save_path = f"checkpoints/{self.model_name}.pt"
        os.makedirs("checkpoints", exist_ok=True)
        
        print(f"\n START TRAINING {self.model_name.upper()}")
        for epoch in range(1, SETTINGS["epochs"] + 1):
            train_loss = self.train_one_epoch()
            val_loss, val_rouge, _, _ = self.evaluate(self.dev_loader)
            
            print(f"Epoch {epoch}: Train Loss={train_loss:.4f} | Val Loss={val_loss:.4f} | ROUGE-L={val_rouge:.4f}")
            
            if val_rouge > best_score:
                best_score = val_rouge
                no_improve = 0
                torch.save(self.model.state_dict(), save_path)
                print("  >>> Saved Best Model!")
            else:
                no_improve += 1
                if no_improve >= SETTINGS["patience"]:
                    print(" Early Stopping ")
                    break
        
        print(f"\n TESTING {self.model_name.upper()} <<<")
        if os.path.exists(save_path):
            self.model.load_state_dict(torch.load(save_path))
            print("  >>> Loaded Best Checkpoint.")
        else:
            print("  >>> Warning: No checkpoint found (Model didn't learn well). Using last epoch state.")

        test_loss, test_rouge, preds, refs = self.evaluate(self.test_loader)
        
        print(f"FINAL TEST ROUGE-L: {test_rouge:.4f}")
        print("\n Example Predictions")
        for i in range(min(3, len(preds))):
            print(f"Ref : {refs[i]}")
            print(f"Pred: {preds[i]}")
            print("-" * 20)

In [42]:
def main():
    # #  Bài 1
    print("\n" + "="*40)
    trainer1 = DirectTrainer("bai1")
    trainer1.run()
    
    # #  Bài 2
    print("\n" + "="*40)
    trainer2 = DirectTrainer("bai2")
    trainer2.run()
    
    #  Bài 3
    print("\n" + "="*40)
    trainer3 = DirectTrainer("bai3")
    trainer3.run()

if __name__ == "__main__":
    main()


[BAI1] Loading Data...
[BAI1] Building Model...

 START TRAINING BAI1


Training:   0%|          | 0/625 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/63 [00:00<?, ?it/s]

Epoch 1: Train Loss=6.3926 | Val Loss=6.3521 | ROUGE-L=0.1694
  >>> Saved Best Model!


Training:   0%|          | 0/625 [00:00<?, ?it/s]

...

Training:   0%|          | 0/625 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/63 [00:00<?, ?it/s]

Epoch 14: Train Loss=4.6267 | Val Loss=5.1364 | ROUGE-L=0.3286
 Early Stopping 

 TESTING BAI3 <<<
  >>> Loaded Best Checkpoint.


Evaluating:   0%|          | 0/63 [00:00<?, ?it/s]

FINAL TEST ROUGE-L: 0.3392

 Example Predictions
Ref : anh albert và chị susan , thuộc hội thánh west ở tuscaloosa , alabama
Pred: một người , khi tôi nghĩ , những người , những người , những người , những người , trong năm .
--------------------
Ref : ngày 11 và 12 - 1 - 2020 , những cơn bão lớn đã quét qua và phá huỷ nhiều vùng ở miền nam và miền trung hoa kỳ .
Pred: những người đã nữ đầu đầu của những người đã dụng những người và sự sống của những người , những người đã bị dụng của những người , và người , .
--------------------
Ref : những trận mưa to và gió lớn trong suốt hai ngày cùng với nhiều cơn lốc xoáy đã gây thiệt hại nặng nề cho nhiều bang .
Pred: một người trong những người , những người , và những người có thể ra những người có thể ra một người .
--------------------
